In [1]:
# maze env

import numpy as np
import time, sys
import tkinter as tk

UNIT = 40 # pixels
MAZE_H = 4 # grid height
MAZE_W = 4 # gird width


class Maze(tk.Tk, object):
    def __init__(self):
        super(Maze, self).__init__()
        self.action_space = ['u','d','l','r']
        self.n_actions = len(self.action_space)
        self.title('Maze')
        self.geometry("{}x{}".format(MAZE_H*UNIT, MAZE_W*UNIT))
        self.__build_maze()
    
    def __build_maze(self):
        self.canvas = tk.Canvas(self, bg = 'white', height = MAZE_H*UNIT, width = MAZE_W*UNIT)
        
        # create grid
        for c in range(MAZE_W*UNIT, UNIT):
            x0,y0, x1,y1 = c,0, c, MAZE_H*UNIT
            self.canvas.create_line(x0,y0,x1,y1)
        for r in range(MAZE_H*UNIT, UNIT):
            x0,y0, x1,y1 = 0,r, MAZE_W*UNIT, r
            self.canvas.create_line(x0,y0,x1,y1)
        
        # create origin
        origin = np.array([20,20])
        
        # hell
        hell1_center = origin + np.array([UNIT*2, UNIT])
        self.hell1 = self.canvas.create_rectangle(
            hell1_center[0]-15, hell1_center[1]-15,
            hell1_center[0]+15, hell1_center[1]+15,
            fill='black'
        )
        
        # hell
        hell2_center = origin + np.array([UNIT, UNIT*2])
        self.hell2 = self.canvas.create_rectangle(
            hell2_center[0]-15, hell2_center[1]-15,
            hell2_center[0]+15, hell2_center[1]+15,
            fill='black'
        )
        
        # create oval
        oval_center = origin + UNIT*2
        self.oval = self.canvas.create_oval(
            oval_center[0]-15, oval_center[1]-15,
            oval_center[0]+15, oval_center[1]+15,
            fill='yellow')
        
        # create red rect
        self.rect = self.canvas.create_rectangle(
            origin[0]-15, origin[1]-15,
            origin[0]+15, origin[1]+15,
            fill='red')
        
        # pack all
        self.canvas.pack()
        
    def reset(self):
        self.update()
        time.sleep(.5)
        self.canvas.delete(self.rect)
        origin = np.array([20,20])
        self.rect = self.canvas.create_rectangle(
            origin[0]-15, origin[1]-15,
            origin[0]+15, origin[1]+15,
            fill='red'
        )
        # return observation
        return self.canvas.coords(self.rect)
    
    def step(self, action):
        s = self.canvas.coords(self.rect)
        base_action = np.array([0,0])
        if action == 0: # up
            if s[1]> UNIT: base_action[1]-= UNIT
        elif action == 1: # down
            if s[1]<(MAZE_H-1)*UNIT:
                base_action[1] += UNIT
        elif action ==2: #right
            if s[0] < (MAZE_W-1)*UNIT:
                base_action[0]+= UNIT
        elif action ==3: # left
            if s[0]>UNIT:
                base_action[0] -= UNIT
        
        self.canvas.move(self.rect, base_action[0], base_action[1]) # move agent
        s_ = self.canvas.coords(self.rect) # next state
        
        # reward function
        if s_ == self.canvas.coords(self.oval):
            reward = 1
            done = True
            s_ = 'terminal'
        elif s_ in [self.canvas.coords(self.hell1), self.canvas.coords(self.hell2)]:
            reward = -1
            done = True
            s_ = "terminal"
        else:
            reward = 0
            done = False
        return s_, reward, done
    
    def render(self):
        time.sleep(.1)
        self.update()
        
def update():
    for t in range(10):
        s = env.reset()
        while True:
            env.render()
            a = 1
            s, r, done = env.step(a)
            if done: break

In [2]:
# Q-learning (on-policy)
import numpy as np
import pandas as pd

class QlearningTable:
    def __init__(self, actions, learning_rate=.001, reward_decay=.9, e_greedy=.9):
        self.actions = actions # a list
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)
        
    def choose_action(self, observation):
        self.check_state_exist(observation)
        # action selection
        if np.random.uniform()<self.epsilon:
            # choose the best action
            state_action = self.q_table.loc[observation, :]
            # some actions may have the same value, randomly choose such actions
            action = np.random.choice(state_action[state_action==np.max(state_action)].index)
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action
    
    def learn(self, s, a, r, s_):
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s,a]
        if s_!="terminal":
            q_target = r + self.gamma*self.q_table.loc[s_,:].max() # when the next state is not terminal
        else:
            q_target = r # the next state is terminal
        self.q_table.loc[s, a] += self.lr* (q_target-q_predict)
        
        
        
    def check_state_exist(self, state):
        if state not in self.q_table.index:
            # append new state to q_table
            self.q_table = self.q_table.append(
                pd.Series([0]*len(self.actions),
                    index = self.q_table.columns,
                    name = state
                )
            )
            

In [3]:
def update():
    for episode in range(100):
        # initial observation
        observation = env.reset()
        
        while True:
            # fresh env
            env.render()
            
            # rl chose action based on an observation
            action = RL.choose_action(str(observation))
            
            # RL takes action and get the next observation and reward
            observation_, reward, done = env.step(action)
            
            # RL learns from this transition
            RL.learn(str(observation), action, reward, str(observation_))
            
            # swap observation
            observation = observation_
            
            # break while loop at the end of the episode
            if done: break
                
    print("Game over")
    env.destroy()

In [6]:
# main
env = Maze()
RL = QlearningTable(actions=list(range(env.n_actions)))
env.after(100, update)
env.mainloop()

Game over
